# NetworkX - Easy Graph Analytics

NetworkX is the most popular library for graph analytics available in Python, or quite possibly any language. To illustrate this, NetworkX was downloaded more than 71 million times in September of 2024 alone, which is roughly 71 times more than the next most popular graph analytics library! [*](https://en.wikipedia.org/wiki/NetworkX) NetworkX has earned this popularity from its very easy-to-use API, the wealth of documentation and examples available, the large (and friendly) community behind it, and its easy installation which requires nothing more than Python.

However, NetworkX users are familiar with the tradeoff that comes with those benefits. The pure-Python implementation often results in poor performance when graph data starts to reach larger scales, limiting the usefulness of the library for many real-world problems.

# Accelerated NetworkX - Easy (and fast!) Graph Analytics

To address the performance problem, NetworkX 3.0 introduced a mechanism to dispatch algorithm calls to alternate implementations. The NetworkX Python API remains the same but NetworkX will use more capable algorithm implementations provided by one or more backends. This approach means users don't have to give up NetworkX -or even change their code- in order to take advantage of GPU performance.

# Let's Get the Environment Setup
This notebook will demonstrate NetworkX both with and without GPU acceleration provided by the `nx-cugraph` backend.

`nx-cugraph` is available as a package installable using `pip`, `conda`, and [from source](https://github.com/rapidsai/nx-cugraph).  Before importing `networkx`, lets install `nx-cugraph` so it can be registered as an available backend by NetworkX when needed.  We'll use `pip` to install.

NOTES:
* `nx-cugraph` requires a compatible NVIDIA GPU, NVIDIA CUDA and associated drivers, and a supported OS. Details about these and other installation prerequisites can be seen [here](https://docs.rapids.ai/install#system-req).
* The `nx-cugraph` package is currently hosted by NVIDIA and therefore the `--extra-index-url` option must be used.
* `nx-cugraph` is supported on specific 11.x and 12.x CUDA versions, and the major version number must be known in order to install the correct build (this is determined automatically when using `conda`).

To find the CUDA major version on your system, run the following command:

In [ ]:
!nvcc --version

From the above output we can see we're using CUDA 12.x so we'll be installing `nx-cugraph-cu12`. If we were using CUDA 11.x, the package name would be `nx-cugraph-cu11`. We'll also be adding `https://pypi.nvidia.com` as an `--extra-index-url`:

In [ ]:
!pip install nx-cugraph-cu12 --extra-index-url=https://pypi.nvidia.com

Of course, we'll also be using `networkx`, which is already provided in the Colab environment. This notebook will be using features added in version 3.3, so we'll import it here to verify we have a compatible version.

In [ ]:
import networkx as nx
nx.__version__

# Let's Start with Something Simple

To begin, we'll compare NetworkX results without a backend to results of the same algorithm using the `nx-cugraph` backend on a small graph.  `nx.karate_club_graph()` returns an instance of the famous example graph consisting of 34 nodes and 78 edges from Zachary's paper, described [here](https://en.wikipedia.org/wiki/Zachary%27s_karate_club).

## Betweenness Centrality
[Betweenness Centrality](https://en.wikipedia.org/wiki/Betweenness_centrality) is a graph algorithm that computes a centrality score for each node (`v`) based on how many of the shortest paths between pairs of nodes in the graph pass through `v`. A higher centrality score represents a node that "connects" other nodes in a network more than that of a node with a lower score.

First, let's create a NetworkX Graph instance of the the Karate Club graph and inspect it.

In [ ]:
G = nx.karate_club_graph()
G.number_of_nodes(), G.number_of_edges()

Next, let's run betweenness centrality and save the results.  Because the Karate Club graph is so small, this should not take long.

In [ ]:
%%time
nx_bc_results = nx.betweenness_centrality(G)

Now, let's run the same algorithm on the same data using the `nx-cugraph` backend.

There are several ways to instruct NetworkX to use a particular backend instead of the default implementation. Here, we will use the `config` API, which was added in NetworkX version 3.3.

The following two lines set the backend to "cugraph" and enable graph conversion caching.

Some notes:
* The standard convention for NetworkX backends is to name the package with a `nx-` prefix to denote that these are packages intended to be used with NetworkX, but the `nx-` prefix is not included when referring to them in NetworkX API calls. Here, `nx-cugraph` is the name of the backend package, and `"cugraph"` is the name NetworkX will use to refer to it.
* NetworkX can use multiple backends! `nx.config.backend_priority` is a list that can contain several backends, ordered based on priority. If a backend in the list cannot run a particular algorithm (either because it isn't supported in the backend, the algorithm doesn't support a particular option, or some other reason), NetworkX will try the next backend in the list. If no specified backend is able to run the algorithm, NetworkX will fall back to the default implementation.
* Many backends have their own data structures for representing an input graph, often optimized for that backend's implementation. Prior to running a backend algorithm, NetworkX will have the backend convert the standard NetworkX Graph instance to the backend-specific type. This conversion can be expensive, and rather than repeat it as part of each algorithm call, NetworkX can cache the conversion so it can be skipped on future calls if the graph doesn't change. This caching can save significant time and improve overall performance.

In [ ]:
nx.config.backend_priority=["cugraph"]  # NETWORKX_BACKEND_PRIORITY=cugraph
nx.config.cache_converted_graphs=True   # NETWORKX_CACHE_CONVERTED_GRAPHS=True

In [ ]:
%%time
nxcg_bc_results = nx.betweenness_centrality(G)

You may have noticed that using the `nx-cugraph` backend resulted in a slightly slower execution time. This is not surprising when working with a graph this small, since the overhead of converting the graph for the first time and launching the algorithm kernel on the GPU is actually significantly more than the computation time itself.  We'll see later that this overhead is negligible when compared to the time saved when running on a GPU for larger graphs.

Since we've enabled graph conversion caching, we can see that if we re-run the same call the execution time is noticeably shorter.

In [ ]:
%%time
nxcg_bc_results = nx.betweenness_centrality(G)

Notice the warning above about using the cache. This will only be raised **once** per graph instance (it can also be easily disabled), but its purpose is to point out that the cache should not be used if the Graph object will have its attribute dictionary modified directly. In this case and many others, we won't be modifying the dictionaries directly. Instead, we will use APIs such as `nx.set_node_attributes` which properly clear the cache, so it's safe for us to use the cache. Because of that, we'll disable the warning so we don't see it on other graphs in this session.

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="Using cached graph for 'cugraph' backend")

Smaller graphs are also easy to visualize with NetworkX's plotting utilities. The flexibility of NetworkX's `Graph` instances make it trivial to add the betweenness centrality scores back to the graph object as node attributes. This will allow us to use those values for the visualization.

In this case, we'll create new attributes for each node called "nx_bc" for the default NetworkX results, and "nxcg_bc" for the nx-cugraph results. We'll use those values to assign the color for each node and plot two graphs side-by-side. This will make it easy to visually validate that the nodes with the higher centrality scores for both implementations match and do indeed appear to be more "central" to other nodes.

In [ ]:
nx.set_node_attributes(G, nx_bc_results, "nx_bc")
nx.set_node_attributes(G, nxcg_bc_results, "nxcg_bc")

In [ ]:
# Configure plot size and layout/position for each node
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
pos = nx.spring_layout(G)

# Assign colors for each set of betweenness centrality results
nx_colors = [G.nodes[n]["nx_bc"] for n in G.nodes()]
nxcg_colors = [G.nodes[n]["nxcg_bc"] for n in G.nodes()]

# Plot the graph and color each node corresponding to NetworkX betweenness centrality values
plt.subplot(1, 2, 1)
nx.draw(G, pos=pos, with_labels=True, node_color=nx_colors)

# Plot the graph and color each node corresponding to nx-cugraph betweenness centrality values
plt.subplot(1, 2, 2)
nx.draw(G, pos=pos, with_labels=True, node_color=nxcg_colors)

As we can see, the same two nodes (`0` and `33`) are the two most central in both graphs, followed by `2`, `31`, and `32`.

## PageRank
Another popular algorithm is [PageRank](https://en.wikipedia.org/wiki/PageRank). PageRank also assigns scores to each node, but these scores are based on analyzing links to each node to determine relative "importance" within the graph.

Let's update the config to use the default NetworkX implementation and run `nx.pagerank`.

In [ ]:
nx.config.backend_priority=[]

In [ ]:
%%time
nx_pr_results = nx.pagerank(G)

We could set `nx.config.backend_priority` again to list `"cugraph"` as the backend, but let's instead show how the `backend` kwarg can be used to override the priority list and force a specific backend to be used.

In [ ]:
%%time
nxcg_pr_results = nx.pagerank(G, backend="cugraph")

In this example, instead of plotting the graph to show that the results are identical, we can compare them directly using the saved values from both runs.

In [ ]:
sorted(nx_pr_results) == sorted(nxcg_pr_results)

# Working with Bigger Data

Now we'll look at a larger dataset from https://snap.stanford.edu/data/cit-Patents.html which contains citations across different U.S. patents granted from January 1, 1963 to December 30, 1999. The dataset represents 16.5M citations (edges) between 3.77M patents (nodes).

This will demonstrate that data of this size starts to push the limits of the default pure-Python NetworkX implementation.

In [ ]:
# The locale encoding may have been modified from the plots above, reset here to run shell commands
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!wget https://data.rapids.ai/cugraph/datasets/cit-Patents.csv  # Skip if cit-Patents.csv already exists.
# !wget https://snap.stanford.edu/data/cit-Patents.txt.gz  # Skip if cit-Patents.txt.gz already exists.

In [ ]:
%load_ext cudf.pandas
import pandas as pd

In [ ]:
%%time
df = pd.read_csv("cit-Patents.csv",
                sep=" ",
                names=["src", "dst"],
                dtype="int32",
)
# df = pd.read_csv("cit-Patents.txt.gz",
#                  compression="gzip",
#                  skiprows=4,
#                  sep="\t",
#                  names=["src", "dst"],
#                  dtype="int32",
# )

In [ ]:
%%time
G = nx.from_pandas_edgelist(df, source="src", target="dst")
G.number_of_nodes(), G.number_of_edges()

By default, `nx.betweenness_centrality` will perform an all-pairs shortest path analysis when determining the centrality scores for each node. However, due to the much larger size of this graph, determining the shortest path for all pairs of nodes in the graph is not feasible. Instead, we'll use the parameter `k` to limit the number of shortest path computations used for determining the centrality scores, at the expense of accuracy. As we'll see when using a dataset this size with `nx.betweenness_centrality`, we have to limit `k` to `1` which is not practical but is sufficient here for demonstration purposes (since anything larger than `1` will result in many minutes of execution time).

In [ ]:
%%time
bc_results = nx.betweenness_centrality(G, k=1)

Now we'll configure NetworkX to use the `nx-cugraph` backend (again, using the name convention that drops the package name's `nx-` prefix) and run the same call. Because this is a Graph that `nx-cugraph` hasn't seen before, the runtime will include the time to convert and cache a GPU-based graph.

In [ ]:
nx.config.backend_priority = ["cugraph"]

In [ ]:
%%time
bc_results = nx.betweenness_centrality(G, k=1)

Let's run betweenness centrality again, now with a more useful number of samples by setting `k=100`.

In [ ]:
%%time
bc_results = nx.betweenness_centrality(G, k=100)

Let's also run pagerank on the same dataset to compare.

In [ ]:
nx.config.backend_priority = []

In [ ]:
%%time
nx_pr_results = nx.pagerank(G)

In [ ]:
%%time
nxcg_pr_results = nx.pagerank(G, backend="cugraph")

In [ ]:
sorted(nx_pr_results) == sorted(nxcg_pr_results)

---
<i>
Information on the U.S. Patent Citation Network dataset used in this notebook is as follows:
<br>Authors: Jure Leskovec and Andrej Krevl
<br>Title: SNAP Datasets, Stanford Large Network Dataset Collection
<br>URL: http://snap.stanford.edu/data
<br>Date: June 2014
</i>
